# Getting started - level 2

In this tutorial we will explore a little bit more advanced example of a program that require some configuration, requirements setup, etc. 

Again we will start with writing code for our program and saving it to [./source_files/gs_level_2.py](./source_files/gs_level_2.py) file.
This time it will be VQE example from [Qiskit documentation](https://qiskit.org/documentation/nature/tutorials/07_leveraging_qiskit_runtime.html) and we also introduce dependency management and arguments to our programs.

```python
# source_files/gs_level_2.py

import argparse

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import QubitConverter
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.circuit.library import EfficientSU2
import numpy as np
from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import SPSA
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator


def run(bond_distance: float = 2.5):
    driver = PySCFDriver(
        atom=f"Li 0 0 0; H 0 0 {bond_distance}",
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    problem = driver.run()

    active_space_trafo = ActiveSpaceTransformer(
        num_electrons=problem.num_particles, num_spatial_orbitals=3
    )
    problem = active_space_trafo.transform(problem)
    qubit_converter = QubitConverter(ParityMapper(), two_qubit_reduction=True)

    ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement="linear", insert_barriers=True)

    np.random.seed(5)
    algorithm_globals.random_seed = 5


    optimizer = SPSA(maxiter=100)
    initial_point = np.random.random(ansatz.num_parameters)

    estimator = Estimator()
    local_vqe = VQE(
        estimator,
        ansatz,
        optimizer,
        initial_point=initial_point,
    )

    local_vqe_groundstate_solver = GroundStateEigensolver(qubit_converter, local_vqe)
    local_vqe_result = local_vqe_groundstate_solver.solve(problem)

    print(local_vqe_result)


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--bond_length",
        help="Bond length in Angstrom.",
        default=2.5,
        type=float,
    )
    args = parser.parse_args()

    print(f"Running for bond length {args.bond_length}.")
    run(args.bond_length)

```

As you can see here we used couple of additional things compared to `getting started level 1`. 

First, we are introducing dependency management by using the `qiskit-nature` module and `pyscf` extension.
We also using argument parsing to accept arguments to our program. In this case argument is `bond_length`. This means that we can, re-run our program over different bond lengths and produce a dissociation curve.


Next we need to run this program. For that we need to import necessary modules and configure `QuantumServerless` client. We are doing so by providing name and host for deployed infrastructure.

In [1]:
from quantum_serverless import QuantumServerless, GatewayProvider

In [2]:
provider = GatewayProvider(
    username="user", # this username has already been defined in local docker setup and does not need to be changed
    password="password123", # this password has already been defined in local docker setup and does not need to be changed
    host="http://gateway:8000", # address of provider
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

In addition to that we will provide additional `dependencies` and `arguments` to our `Program` construction.
- `dependencies` parameter will install provided libraries to run our script. Dependencies can be python libraries available on PyPi or any package source installable via pip package manager .
- `arguments` parameter is a dictionary with arguments that will be passed for script execution

In [3]:
from quantum_serverless import Program

program = Program(
    title="Getting started program level 2",
    entrypoint="gs_level_2.py",
    working_dir="./source_files",
    dependencies=["qiskit-nature", "qiskit-nature[pyscf]"],
    arguments={
        "bond_length": 2.55
    }
)

job = serverless.run_program(program)
job

<Job | 40754290-9768-4ba2-9689-6d72e9197878>

In [8]:
job.status()

'SUCCEEDED'

In [9]:
print(job.logs())

Running for bond length 2.55.
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.211426461751
  - computed part:      -8.211426461751
  - ActiveSpaceTransformer extracted energy part: 0.0
~ Nuclear repulsion energy (Hartree): 0.622561424612
> Total ground state energy (Hartree): -7.588865037139
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 3.997 S: 0.436 S^2: 0.626 M: 0.001
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  4.81880162]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.53218981]
    - computed part:      [0.0  0.0  1.53218981]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  3.28661181]  Total: 3.28661181
                 (debye): [0.0  0.0  8.35373319]  Total: 8.35373319
 



---
If you want to run this program with different bond length you can run it 3 times. Programs are asynchronous, therefore each of instance of program will be running in parallel.

In [10]:
jobs = []

for bond_length in [2.55, 3.0, 3.55]:
    program = Program(
        title=f"Groundstate with bond length {bond_length}",
        entrypoint="gs_level_2.py",
        working_dir="./source_files",
        dependencies=["qiskit-nature", "qiskit-nature[pyscf]"],
        arguments={
            "bond_length": bond_length
        }
    )
    jobs.append(serverless.run_program(program))

jobs

[<Job | ddb7331c-6db2-4e44-9449-b58ba53bd6b8>,
 <Job | 52f77d9d-c669-4526-8b33-2fa9290b71dd>,
 <Job | c31428f3-8c15-4257-a53d-fd899d40e29f>]

In [14]:
for job in jobs:
    print(job.status())

RUNNING
RUNNING
RUNNING


In [10]:
for job in jobs:
    print(job.logs())

Running for bond length 2.55.
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.211426461751
  - computed part:      -8.211426461751
  - ActiveSpaceTransformer extracted energy part: 0.0
~ Nuclear repulsion energy (Hartree): 0.622561424612
> Total ground state energy (Hartree): -7.588865037139
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 3.997 S: 0.436 S^2: 0.626 M: 0.001
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  4.81880162]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.53218981]
    - computed part:      [0.0  0.0  1.53218981]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  3.28661181]  Total: 3.28661181
                 (debye): [0.0  0.0  8.35373319]  Total: 8.35373319
 

Running for bond length 3.0.
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.124024370249
  - computed part:      -8.124024370249
  - ActiveSpaceTransformer

---
Other way would be refactoring program file itself to accept list of bond length and run them in a loop inside a program.
If you want 3 independent results, then running 3 programs would be a better fit. But if you want to do some postprocessing after execution of multiple function, then refactoring program file to run 3 function and postprocess them would be better choice. But at the end it all boils down to user preference.